In [16]:
import pandas as pd
import numpy as np
import os

In [17]:
csvfile_1 = "Hawaii_Measures_Clean.csv"
csvfile_2 = "Hawaii_Stations_Clean.csv"

In [18]:
measure = pd.read_csv(csvfile_1, dtype = object)
station = pd.read_csv(csvfile_2, dtype = object)
measure = measure.drop(['Unnamed: 0'], axis = 1)
station = station.drop(['Unnamed: 0'], axis = 1)
measure.head()

station        date  prcp tobs
0  USC00519397  2010-01-01  0.08   65
1  USC00519397  2010-01-02   0.0   63
2  USC00519397  2010-01-03   0.0   74
3  USC00519397  2010-01-04   0.0   76
4  USC00519397  2010-01-07  0.06   70

In [19]:
clean_measurement = "cleaned_measurement.csv"
clean_station = "cleaned_station.csv"
measure.to_csv(clean_measurement, index=False)
station.to_csv(clean_station, index = False)

In [43]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from datetime import datetime

In [44]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [45]:
conn = engine.connect()

In [46]:
Base = declarative_base()
    
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key = True)
    date = Column(DateTime)
    prcp = Column(Float)
    station = Column(Float)
    tobs = Column(Float)
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [47]:
Base.metadata.create_all(engine)

In [48]:
df_of_data_to_insert_1 = pd.read_csv(clean_measurement)
df_of_data_to_insert_2 = pd.read_csv(clean_station)
df_of_data_to_insert_1.head()

station        date  prcp  tobs
0  USC00519397  2010-01-01  0.08    65
1  USC00519397  2010-01-02  0.00    63
2  USC00519397  2010-01-03  0.00    74
3  USC00519397  2010-01-04  0.00    76
4  USC00519397  2010-01-07  0.06    70

In [49]:
data_1 = df_of_data_to_insert_1.to_dict(orient='records')
data_2 = df_of_data_to_insert_2.to_dict(orient='records')
data_1[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [50]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [51]:
table_1 = sqlalchemy.Table('measurement', metadata, autoload=True)
table_2 = sqlalchemy.Table('station', metadata, autoload=True) 

In [54]:
conn.execute(table_1.insert(), data_1)
conn.execute(table_2.insert(), data_2)

StatementError: (builtins.TypeError) SQLite Date type only accepts Python date objects as input. [SQL: 'INSERT INTO measurement (station, date, prcp, tobs) VALUES (?, ?, ?, ?)'] [parameters: [{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}, {'station': 'USC00519397', 'date': '2010-01-08', 'prcp': 0.0, 'tobs': 64}, {'station': 'USC00519397', 'date': '2010-01-09', 'prcp': 0.0, 'tobs': 68}, {'station': 'USC00519397', 'date': '2010-01-10', 'prcp': 0.0, 'tobs': 73}  ... displaying 10 of 18103 total bound parameter sets ...  {'station': 'USC00516128', 'date': '2017-08-22', 'prcp': 0.5, 'tobs': 76}, {'station': 'USC00516128', 'date': '2017-08-23', 'prcp': 0.45, 'tobs': 7

In [53]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', None, None, None, None),
 (2, 'USC00519397', None, None, None, None),
 (3, 'USC00519397', None, None, None, None),
 (4, 'USC00519397', None, None, None, None),
 (5, 'USC00519397', None, None, None, None)]